In [1]:
import geohash2
import pandas as pd
import requests
class crawling_apt:

    def __init__(self, addr):
        self.addr = addr
        self.apt_data = None

    def get_addr(self):
        
        url="https://apis.zigbang.com/v2/search?q={}&serviceType=아파트".format(self.addr)
        response = requests.get(url)
        data=response.json()
        lat_lng_list=[(data["items"][i]["lat"],data["items"][i]["lng"]) for i in range(len(data["items"]))]
        geohash_list= list(set([geohash2.encode(i[0],i[1],5) for i in lat_lng_list]))
        apt_data=[]
        for geo in geohash_list:
            url_="https://apis.zigbang.com/property/apartments/location/v3?e=&geohash={}\
&markerType=large&n=&q=type%3Dsales%2Cprice%3D0~-1%2CfloorArea%3D0~-1&s=&serviceType%5B0%5D=apt&serviceType%5B1%5D=offer&w=".format(geo)
            response = requests.get(url_)
            datas = response.json()
            for data in datas["filtered"]:
                apt_data.append(data)
        self.apt_data = apt_data
        return self.apt_data
        
    
        

    def datafame(self):
        apt_data = self.apt_data
        df=pd.DataFrame(apt_data)[["name","sido","gugun","dong","households","buildDate","item_count","price",'floorArea']]
        df=df[df["gugun"]==self.addr]
        df["rent_avg"]=df["price"].apply(lambda x : x["rent"]["avg"])
        df["rent_per"]=df["price"].apply(lambda x : x["rent"]["perArea"])
        df["sale_avg"]=df["price"].apply(lambda x : x["sales"]["avg"])
        df["slae_per"]=df["price"].apply(lambda x : x["sales"]["perArea"])
        df["item_count"]=df["item_count"].apply(lambda x : x["register"])
        df["floorArea"]=df["floorArea"].apply(lambda x : x["p"])
        df["buildDate"]=df["buildDate"].fillna("0000").apply(lambda x : int(str(x)[0:4]))
        df=df.drop("price",axis=1).fillna(0)
        return df
        self.datafame()